In [1]:
git_dir = '/mnt/c/Users/ArcaroLab/Desktop/git_repos/dhcp'
import sys
#add git_dir to path
sys.path.append(git_dir)
from nilearn import plotting, image
import numpy as np
import nibabel as nib
import dhcp_params as params
import pandas as pd
from nilearn.maskers import NiftiMasker
from glob import glob as glob
from nilearn.maskers import MultiNiftiMasker
from nilearn.maskers import NiftiLabelsMasker
import dhcp_params as params
import matplotlib.pyplot as plt


In [20]:
diffusion_dir = '/mnt/e/diffusion/FinalData/Seeds/40week/distcorr_normmax/V1_lh'

#glob all the files
files = glob(diffusion_dir + '/*.nii.gz')

#extract the subs from each file's name
#find where sub- starts
subs = []
for file in files:
    start = file.find('sub-')
    subs.append(file[start:(start+15)])

#convert to dataframe
df = pd.DataFrame(subs, columns=['subs'])

#keep only unique subs
df = df.drop_duplicates()


In [21]:
#load dhcp sub_list
full_sub_list = pd.read_csv(f'{params.out_dir}/participants.csv')



In [22]:
#add 1 to the sub_list if they are in the diffusion data
full_sub_list['to_run'] = 0
for i in range(len(full_sub_list)):
    if full_sub_list['participant_id'][i] in df['subs'].values:
        full_sub_list['to_run'][i] = 1

/tmp/ipykernel_875347/2364714121.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_sub_list['to_run'][i] = 1
/tmp/ipykernel_875347/2364714121.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_sub_list['to_run'][i] = 1
/tmp/ipykernel_875347/2364714121.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_sub_list['to_run'][i] = 1
/tmp/ipykernel_875347/2364714121.py:5: SettingWithCopyWarning: 
A value is trying to be set

In [ ]:
#save the new sub_list
full_sub_list.to_csv(f'{git_dir}/participants.csv', index=False)

In [7]:
len('sub-CC00063AN06')

15

In [9]:
'''diffusion tests'''
sub = 'sub-CC00063AN06'
ses = 'ses-15102'
roi = 'V1_lh'

raw_data_dir, raw_anat_dir, raw_func_dir, out_dir, anat_suf, func_suf, brain_mask_suf, group_template,template_name = params.load_group_params('infant')
dti_dir = f'/mnt/e/diffusion/'


anat = image.load_img(f'{out_dir}/{sub}/{ses}/anat/{sub}_{ses}_{anat_suf}_brain.nii.gz')

dti_img = image.load_img(f'{dti_dir}/FinalData/Paths/native/distcorr/{roi}/{roi}_{sub}_fdt_paths.nii.gz')

In [5]:
dti_img.shape

(128, 128, 64)

In [7]:
anat.shape

(217, 290, 290)

In [ ]:
#resample dti to anat
dti_img_resamp = image.resample_to_img(dti_img, anat)

#save resampled dti
dti_img_resamp.to_filename(f'{dti_dir}/vlad/{roi}_{sub}_fdt_paths_resamp.nii.gz')

In [3]:
atlas = 'wang'
atlas_name, roi_labels = params.load_atlas_info(atlas)


In [4]:
sub = 'sub-CC00056XX07'
ses = 'ses-10700'

anat_dir = f'{params.raw_anat_dir}/{sub}/{ses}/anat'
func_dir = f'{params.raw_func_dir}/{sub}/{ses}func'
out_dir = f'{params.out_dir}/{sub}/{ses}'
roi_dir = f'{out_dir}/rois/'
#load anatomical
anat_img = image.load_img(f'{anat_dir}/{sub}_{ses}_{params.anat_suf}.nii.gz')
#load functional
func_img = image.load_img(f'{params.out_dir}/{sub}/{ses}/func/{sub}_{ses}_{params.func_suf}_1vol.nii.gz')

stat_img = image.load_img(f'{out_dir}/derivatives/thalmocortical/rh_IPS0_second_order_anat.nii.gz')


In [23]:
#loop through rois and load them


for hemi in ['lh','rh']:
    roi_imgs = []

    
    for roi in roi_labels['label']:
        roi_imgs.append(image.load_img(f'{roi_dir}/{hemi}_{roi}_epi.nii.gz').get_fdata())

    #create a zeros array with the same shape as the functional image
    zero_imgs = np.zeros(func_img.shape)  
    #insert zeros array as the first element of the roi_imgs list
    roi_imgs.insert(0, zero_imgs)
    #conert to numpy array
    roi_imgs = np.array(roi_imgs)
    #insert zeros array as the first element of the roi_imgs array
    

    #for every voxel, get the index of the max value
    max_roi = np.argmax(roi_imgs, axis=0)
    #add 1 to the index to get the roi label
    
    #convert max_roi to nifti
    max_roi = nib.Nifti1Image(max_roi, func_img.affine, func_img.header)

    #save it
    nib.save(max_roi, f'{out_dir}/atlas/Wang_maxprob_surf_{hemi}_edits_epi.nii.gz')


#### Adult


In [42]:
sub = 'sub-104416'
ses = 'ses-01'
#load anatomical
anat_img = image.load_img(f'{params.out_dir}/{sub}/{ses}/anat/{sub}_{ses}_{params.anat_suf}.nii.gz')
#load functional
func_img = image.load_img(f'{params.raw_func_dir}/{sub}/{ses}/func/{sub}_{ses}_{params.func_suf}.nii.gz')
func_dir = f'{params.raw_func_dir}/{sub}/{ses}'
out_dir = f'{params.out_dir}/{sub}/{ses}'
roi_dir = f'{out_dir}/rois/{atlas}'

roi_img = image.load_img(f'{roi_dir}/lh_V1v_epi.nii.gz')

atlas_dir = f'{out_dir}/atlas/Wang_maxprob_surf_lh_edits_epi.nii'

In [47]:
masker = NiftiLabelsMasker(
    labels_img=atlas_dir,
    standardize="zscore_sample",
    standardize_confounds="zscore_sample",
    smoothing_fwhm=params.smooth_mm)

In [40]:
#glob all func files
func_files = glob(f'{func_dir}/func/*_bold.nii.gz')
func_files = func_files[0:1]

In [48]:
time_series = masker.fit_transform(func_img)

In [49]:
time_series.shape

(900, 25)

In [46]:
time_series[:,0]


array([ 2.10849380e+00,  1.36608434e+00,  6.96354806e-01,  1.28054440e-01,
       -7.27727190e-02,  2.54483998e-01,  1.97983578e-01, -4.27351110e-02,
       -2.87330359e-01, -4.64480281e-01, -1.92493707e-01,  1.01197019e-01,
        1.13732032e-01,  3.94226164e-01,  4.68739867e-01,  8.09309185e-01,
        1.16738117e+00,  1.33421290e+00,  1.48876500e+00,  1.21987736e+00,
        9.55551028e-01,  7.80397415e-01,  1.80097967e-01,  1.80364907e-01,
        4.67997044e-01,  9.04273510e-01,  1.15081871e+00,  9.00872827e-01,
        1.02777827e+00,  8.59797478e-01,  8.60017955e-01,  1.19642055e+00,
        1.42938614e+00,  1.54112184e+00,  1.19835889e+00,  1.64164567e+00,
        1.37216616e+00,  1.67436445e+00,  1.59777319e+00,  1.18567300e+00,
        9.60321307e-01,  8.27334106e-01,  9.85936821e-01,  9.50989664e-01,
        1.08010030e+00,  1.27908206e+00,  7.37488210e-01,  1.17102563e+00,
        1.35952663e+00,  1.85706222e+00,  1.39418364e+00,  1.81953847e+00,
        1.94103515e+00,  

In [28]:
#glob all func files
func_files = glob(f'{func_dir}/func/*_bold.nii.gz')


In [29]:

        

#extract roi timeseries
masker = MultiNiftiMasker(mask_img = roi_img, standardize = True, smoothing_fwhm=params.smooth_mm)

In [30]:
fmri_data = masker.fit_transform(func_files)

/home/treeshrew1/anaconda3/envs/fmri/lib/python3.9/site-packages/joblib/memory.py:353: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  return self.func(*args, **kwargs)


In [31]:
len(fmri_data)

4

In [32]:
fmri_data[1].shape

(900, 1452)

In [6]:

func_files = glob(f'{func_dir}/func/*_bold.nii.gz')

#loop through func files
all_func = []
for n, func_file in enumerate(func_files):
    print(n)
    #load func
    try:
        print(f'Loading {func_file}')
        curr_img = image.load_img(func_file)
        
    except:
        print(f'Error loading {func_file}')
        continue
    
    all_func.append(curr_img)

    

#concatenate all func files
func_img = image.concat_imgs(all_func)

0
Loading /mnt/f/7T_HCP/sub-100610/ses-01/func/sub-100610_ses-01_task-rest_run-01_preproc_bold.nii.gz
1
Loading /mnt/f/7T_HCP/sub-100610/ses-01/func/sub-100610_ses-01_task-rest_run-02_preproc_bold.nii.gz
2
Loading /mnt/f/7T_HCP/sub-100610/ses-01/func/sub-100610_ses-01_task-rest_run-03_preproc_bold.nii.gz
3
Loading /mnt/f/7T_HCP/sub-100610/ses-01/func/sub-100610_ses-01_task-rest_run-04_preproc_bold.nii.gz
Error loading /mnt/f/7T_HCP/sub-100610/ses-01/func/sub-100610_ses-01_task-rest_run-04_preproc_bold.nii.gz


In [7]:
func_img = image.concat_imgs(all_func)

: 

In [21]:
roi_img = image.load_img(f'{params.out_dir}/{sub}/{ses}/rois/{atlas}/lh_V1v_epi.nii.gz')
masker = NiftiMasker(mask_img = roi_img, standardize = True, smoothing_fwhm=params.smooth_mm)

In [22]:
curr_ts = masker.fit_transform(func_img)

/home/treeshrew1/anaconda3/envs/fmri/lib/python3.9/site-packages/joblib/memory.py:353: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  return self.func(*args, **kwargs)


In [23]:
#convert to numpy 
roi_np = roi_img.get_fdata()
func_np = func_img.get_fdata()

#multiply roi and func
roi_func = np.multiply(roi_np, func_np)

#extract roi timeseries
roi_ts = roi_func[roi_func != 0]

ValueError: operands could not be broadcast together with shapes (113,136,113) (113,136,113,900) 

In [6]:
anat_affine == func_affine

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [4]:
anat_dir = '/mnt/e/dHCP_raw/rel3_dhcp_anat_pipeline'
func_dir = '/mnt/e/dHCP_raw/rel3_dhcp_fmri_pipeline'

data_dir = '/mnt/e/dhcp_analysis_full'
sub = 'sub-CC00056XX07'
ses= 'ses-10700'
sub_dir = f'{data_dir}/{sub}/{ses}'
sub_anat = f'{anat_dir}/{sub}/{ses}'
sub_func = f'{func_dir}/{sub}/{ses}'
roi_dir = f'{sub_dir}/rois'
atlas_dir = f'{sub_dir}/atlas'



In [7]:
anat = image.load_img(f'{sub_anat}/anat/{sub}_{ses}_desc-restore_T2w.nii.gz')
affine = anat.affine

func = image.load_img(f'{sub_dir}/func/{sub}_{ses}_task-rest_desc-preproc_bold_1vol.nii.gz')
atlas = image.load_img(f'{atlas_dir}/calcsulc_binnedroi_rh_anat.nii.gz')
atlas = image.load_img(f'{atlas_dir}/Wang_maxprob_surf_rh_edits_anat.nii')

In [17]:
roi = image.load_img(f'{sub_dir}/atlas/Wang_maxprob_surf_lh_edits_anat.nii')



In [8]:
func.shape

(68, 67, 45, 2300)

In [6]:

#convert to numpy
roi_data = roi.get_fdata()

#extrat mask dimension from roi
roi_data = roi_data[:,:,:,:,1]

#drop last dimension
roi_data = np.squeeze(roi_data)

#convert back to nifti
roi_nifti = nib.Nifti1Image(roi_data, affine=affine)

#save nifti
nib.save(roi_nifti, 'wang_lh.nii.gz')

IndexError: too many indices for array: array is 3-dimensional, but 5 were indexed

In [40]:
roi = image.load_img(f'{sub_dir}/Wang_maxprob_surf_lh_edits_extended_epi.nii')

#create zero array with same dimensions as anat
anat_copy = np.zeros(anat.shape)

#convert roi to numpy
affine = roi.affine

roi_data = roi.get_fdata()



#convert roi_data to nifti
#roi_nifti = nib.Nifti1Image(roi_data, affine)


In [43]:
anat_copy[:,0,0] = roi_data[0, 0, :,0,1]


In [30]:
#convert roi to numpy
affine = np.array(anat.affine)

roi_data = roi.get_fdata()

#extract second dimension
roi_data = roi_data[:,:,:,:,1]

#reshape roi_data to match anat 
roi_data = roi_data.reshape(anat.shape[0],anat.shape[1],anat.shape[2])

#convert roi_data to nifti
roi_nifti = nib.Nifti1Image(roi_data, affine)


In [31]:
roi_nifti.shape

(217, 290, 290)

In [15]:
anat.shape

(217, 290, 290)